In [ ]:
!pip install kaggle
!pip install tensorflow

In [ ]:
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import os
import tensorflow as tf
import shutil
import PIL
import PIL.Image
from sklearn.model_selection import train_test_split
import sklearn
import tensorflow_datasets as tfds

from tensorflow.keras.utils import image_dataset_from_directory
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
import kagglehub

In [ ]:
from google.colab import files
files.upload()


In [ ]:
if not os.path.exists('/root/.kaggle'):
  os.mkdir('/root/.kaggle')
!mv kaggle.json /root/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# import dataset fashion_mnsit
#import kagglehub

# Download latest version
path = kagglehub.dataset_download("prasunroy/natural-images")

print("Path to dataset files:", path)

In [ ]:
dataset_dir = os.path.join(path, "natural_images")

print("path: ", os.listdir(path))

class_names = ['airplane', 'car', 'cat', 'dog', 'flower', 'fruit', 'motorbike', 'person']
y_train = {class_elm: i for i, class_elm in enumerate(class_names)}

train_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    color_mode='rgb',
    image_size=(224,224),
    batch_size=32
)

val_ds = image_dataset_from_directory(
    dataset_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    color_mode='rgb',
    image_size=(224,224),
    batch_size=32
)

val_batches = tf.data.experimental.cardinality(val_ds)
test_ds = val_ds.take(val_batches // 2)
val_ds = val_ds.skip(val_batches // 2)


In [ ]:
train_ds = train_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

In [ ]:
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3), name="vgg16")

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Dense, Flatten#, Sequential
#from

base_model.trainable = False
model = keras.Sequential([
    base_model,
    Flatten(),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(8, activation='softmax')
])

model.compile(loss=SparseCategoricalCrossentropy(from_logits=True), optimizer= Adam(), metrics=['accuracy'])


In [ ]:
train_model = model.fit(train_ds, batch_size=100, epochs=10, validation_data= val_ds)

In [ ]:
loss_test, accuracy_test = model.evaluate(val_ds)
print("loss: ", loss_test)
print("accuracy: ", accuracy_test)


In [ ]:
for b, l in train_ds.take(5):
  pos_pred = model.predict(b)
  predict_label = np.argmax(pos_pred, axis=1)

for bh, lb in test_ds.take(5):
  lab_ans = lb.numpy()


print("pred label: ", predict_label)
print("real label: ", lab_ans)




